Import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
from glob import glob
import IPython.display as ipd
import tensorflow as tf
from PIL import Image
from matplotlib.pyplot import imread
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.utils import to_categorical
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

Import metadata

In [3]:
df = pd.read_csv("ouiseau/bird_songs_metadata.csv")
class_names = df["name"].unique()


getting all the wavefiles

In [4]:
path_to_wav = "ouiseau/wavfiles/"
datafiles = glob(path_to_wav + "*")

Generate spectrograms

In [5]:
def generate_spectrogram(file_audio, identifier):
    audio_data, sample_rate = librosa.load(path_to_wav + file_audio)
    spec_mel = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    spec_mel = librosa.power_to_db(spec_mel, ref=np.max)
    figure, axis = plt.subplots(figsize=(15, 5))
    axis.set_title("Mel Spectrogram")
    plt.suptitle(identifier)
    librosa.display.specshow(spec_mel, x_axis='time', y_axis='log', ax=axis)
    return ipd.Audio(path_to_wav + file_audio, rate=sample_rate)

Process audios

In [6]:
def process_audio(audio_file):
    audio_data, sample_rate = librosa.load(audio_file, duration=10)
    mel_spec = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate) 
    mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec

Generate a dataframe for testing

In [7]:
df_train = pd.DataFrame({"name": df["name"], "audiopath": path_to_wav + df["filename"]})

# Assuming `process_audio` is a function that generates mel spectrograms
df_train["mel_spec"] = df_train["audiopath"].apply(lambda x: process_audio(x))

# Using factorize to encode class labels
df_train["class"] = df_train["name"].factorize()[0]

Shuffle data

In [8]:
df_train = shuffle(df_train)

Separate labels from data

In [9]:
(test_x, test_y) = df_train["mel_spec"].values, df_train["class"].values
test_y = to_categorical(test_y, num_classes=len(class_names))
test_x = np.stack(test_x[:])
test_x = tf.keras.utils.normalize(test_x)
test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_dataset = test_dataset.batch(10)
test_dataset = test_dataset.prefetch(tf.data.AUTOTUNE)

Import model

In [15]:
saved_model_path = "modeles/model84.keras"
model = load_model(saved_model_path)

Test model

In [16]:
model.evaluate(test_dataset)

543/543 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9675 - loss: 0.2508 - precision_2: 0.9678 - recall_2: 0.9674


[0.2144334316253662, 0.9690151214599609, 0.9699095487594604, 0.969199538230896]